In [1]:
import time
import json
import sys
import os

from globus_automate_client import (create_flows_client, graphviz_format, state_colors_for_log,
                                    create_action_client, 
                                    create_flows_client)

In [2]:
CLIENT_ID = "e6c75d97-532a-4c88-b031-8584a319fa3e"

In [3]:
flow_definition = {
  "Comment": "An analysis flow",
  "StartAt": "Transfer",
  "States": {
    "Transfer": {
      "Comment": "Initial transfer",
      "Type": "Action",
      "ActionUrl": "https://actions.automate.globus.org/transfer/transfer",
      "ActionScope": "https://auth.globus.org/scopes/actions.globus.org/transfer/transfer",
      "Parameters": {
        "source_endpoint_id.$": "$.input.source_endpoint", 
        "destination_endpoint_id.$": "$.input.dest_endpoint",
        "transfer_items": [
          {
            "source_path.$": "$.input.source_path",
            "destination_path.$": "$.input.dest_path",
            "recursive": False
          }
        ]
      },
      "ResultPath": "$.Transfer1Result",
      "WaitTime": 600,
      "End": True
    },
  }
}

In [4]:
flows_client = create_flows_client(CLIENT_ID)
flow = flows_client.deploy_flow(flow_definition, title="Simple data transfer")
flow_id = flow['id']

print(flow)

flow_scope = flow['globus_auth_scope']

print(f'Newly created flow with id:\n{flow_id}\nand scope:\n{flow_scope}')

GlobusHTTPResponse({'action_url': 'https://flows.globus.org/flows/b4692aa2-6e4a-431d-bec1-8ec0354461e3', 'administered_by': [], 'api_version': '1.0', 'definition': {'Comment': 'An analysis flow', 'StartAt': 'Transfer', 'States': {'Transfer': {'ActionScope': 'https://auth.globus.org/scopes/actions.globus.org/transfer/transfer', 'ActionUrl': 'https://actions.automate.globus.org/transfer/transfer', 'Comment': 'Initial transfer', 'End': True, 'Parameters': {'destination_endpoint_id.$': '$.input.dest_endpoint', 'source_endpoint_id.$': '$.input.source_endpoint', 'transfer_items': [{'destination_path.$': '$.input.dest_path', 'recursive': False, 'source_path.$': '$.input.source_path'}]}, 'ResultPath': '$.Transfer1Result', 'Type': 'Action', 'WaitTime': 600}}}, 'description': '', 'globus_auth_scope': 'https://auth.globus.org/scopes/b4692aa2-6e4a-431d-bec1-8ec0354461e3/flow_b4692aa2_6e4a_431d_bec1_8ec0354461e3_user', 'globus_auth_username': 'b4692aa2-6e4a-431d-bec1-8ec0354461e3@clients.auth.globu

In [11]:
theta_fx_endpoint = 'f765db7a-038c-47ea-9176-d81de31c054f' #FuncX endpoint

src_endpoint = '9c9cb97e-de86-11e6-9d15-22000a1e3b52' #'aps#data' # Voyager
dest_endpoint = '08925f04-569f-11e7-bef8-22000b9a448b' #'alcf#dtn_theta' # Theta DTN

src_filepath = '/gdata/RAVEN/bicer/2020-1/comm_33IDD/extracted_tekin'
src_filename = 'extracted_scan213.h5'
src_result_path = '/gdata/RAVEN/bicer/2020-1/comm_33IDD/globus_automate_results'

dest_filepath = '/lus/grand/projects/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/transferred'
dest_resultpath = '/lus/grand/projects/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/result'

flow_input = {
    "input": {
        "source_endpoint": f"{src_endpoint}",
        "source_path": f"{src_filepath}/{src_filename}",
        "dest_endpoint": dest_endpoint,
        "dest_path": f"{dest_filepath}/{src_filename}",
    }
}

print(f"transfer file from {src_endpoint}#{src_filepath}/{src_filename} to {dest_endpoint}#{dest_filepath}/")

transfer file from 9c9cb97e-de86-11e6-9d15-22000a1e3b52#/gdata/RAVEN/bicer/2020-1/comm_33IDD/extracted_tekin/extracted_scan213.h5 to 08925f04-569f-11e7-bef8-22000b9a448b#/lus/grand/projects/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/transferred/


In [12]:
flow_action = flows_client.run_flow(flow_id, flow_scope, flow_input)
print(flow_action)

flow_action_id = flow_action['action_id']

flow_status = flow_action['status']
print(f'Flow action started with id: {flow_action_id}')


while flow_status == 'ACTIVE':
    time.sleep(10)
    flow_action = flows_client.flow_action_status(flow_id, flow_scope, flow_action_id)
    flow_status = flow_action['status']
    print(f'Flow status: {flow_status}')

GlobusHTTPResponse({'action_id': 'efa833ae-6610-41c1-8b73-3e7b35666929', 'completion_time': 'None', 'created_by': 'urn:globus:auth:identity:a478e9c0-d274-11e5-9901-63aaec352b80', 'details': {'code': 'ActionStarted', 'description': 'State Transfer of type Action started', 'details': {'input': {'destination_endpoint_id': '08925f04-569f-11e7-bef8-22000b9a448b', 'source_endpoint_id': '9c9cb97e-de86-11e6-9d15-22000a1e3b52', 'transfer_items': [{'destination_path': '/lus/grand/projects/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/transferred/extracted_scan213.h5', 'recursive': False, 'source_path': '/gdata/RAVEN/bicer/2020-1/comm_33IDD/extracted_tekin/extracted_scan213.h5'}]}, 'state_name': 'Transfer', 'state_type': 'Action'}, 'time': '2021-02-03T19:04:08.966000+00:00'}, 'start_time': '2021-02-03T19:04:08.939000+00:00', 'status': 'ACTIVE'})
Flow action started with id: efa833ae-6610-41c1-8b73-3e7b35666929
Flow status: ACTIVE
Flow status: SUCCEEDED


In [13]:
flow_action

GlobusHTTPResponse({'action_id': 'efa833ae-6610-41c1-8b73-3e7b35666929', 'completion_time': '2021-02-03T19:04:29.662000+00:00', 'created_by': 'urn:globus:auth:identity:a478e9c0-d274-11e5-9901-63aaec352b80', 'details': {'output': {'Transfer1Result': {'action_id': '1R2DY8jrQYDee', 'completion_time': '2021-02-03T19:04:29.455344+00:00', 'creator_id': 'urn:globus:auth:identity:a478e9c0-d274-11e5-9901-63aaec352b80', 'details': {'bytes_checksummed': 0, 'bytes_transferred': 123438064, 'canceled_by_admin': None, 'canceled_by_admin_message': None, 'command': 'API 0.10', 'completion_time': '2021-02-03T19:04:22+00:00', 'deadline': '2021-02-04T19:04:13+00:00', 'delete_destination_extra': False, 'destination_endpoint': 'alcf#08925f04-569f-11e7-bef8-22000b9a448b', 'destination_endpoint_display_name': 'alcf#dtn_theta', 'destination_endpoint_id': '08925f04-569f-11e7-bef8-22000b9a448b', 'directories': 0, 'effective_bytes_per_second': 12918862, 'encrypt_data': False, 'event_list': [{'DATA_TYPE': 'event',